In [1]:
import pandas as pd
import plotly.graph_objects as go
from collections import Counter


In [12]:
df = pd.read_pickle('/home/ifte/alechat_core/corpus/model/model_features.pkl')

In [14]:
df['features'].iloc[0]

'| orca australia portfolio page template default term us services solutions portfolio blog contact link portal qo de them ever orca australians one goal one team qo de them orca australia portal connects alumni fb tweet g+ client old rajshahi globally leave com locally help a common goal space g+ right animation image evelyn template team our society us transitions road share tweet application page community rajshahi solutions inv bangladesh nsw bangladesh house width loaded service animation australia b association australia place blog us email dhaka website mac qua rie e us reside college development rajshahi cadet college email fade out animation use clients blog contact single, single portfolio page reserve composer #, road# mac qua rie fields connect comp sector orca australia info@ australians not date ajax fade take single responsive orca australia cadet alumnus responsive association australia incorporated website http a community portal www privacy policy career © more links 

In [17]:
df = pd.read_csv('/home/ifte/resource/docomo_dic/data/chat_history.csv')

In [2]:
df = pd.read_pickle('/home/ifte/resource/docomo_dic/data/cluster/cluster_stats.pkl')

In [4]:
df.head()

,cluster_ID,text,processed
0,8,獲得したポイントは\nいつ反映されますか,獲得 ポイント タイミング 反映
1,8,ポイント獲得タイミング,ポイント 獲得 タイミング
2,8,ポイント獲得タイミング,ポイント 獲得 タイミング
3,2,温泉宿,温泉 旅館
4,2,費用についｔ についｔえ教えて,費用


In [5]:
df = df[df['processed'].str.contains('d')]

In [6]:
record_top3 = df.groupby('cluster_ID')['text'].count().reset_index(name="Count")
record_top3 = record_top3.sort_values(by='Count', ascending=False)
record_top3 = record_top3.cluster_ID.values.tolist()[:3]

In [7]:
record_top3

[2, 3, 15]

In [8]:
collector = []
for idx in record_top3:
    collector.append(df[(df['cluster_ID'] == idx)].text.values.tolist())

In [28]:
filt = []
for s, c in Counter(collector[0]).most_common():
    if c > 1:
        filt.append((s, c))

In [ ]:
filt

In [10]:
sents = 

In [42]:
x=list(set([i for j in collector for i in j]))
flag = 1
i = 0 
fig = go.Figure()
for items in record_top3:
    sn = []
    for s, c in Counter(collector[i]).most_common():
        if c > 2:
            sn.append((s, c))
        
    if flag:
        fig.add_trace(go.Bar(x=x, y=[j for i, j in sn], name=items, width=3, ))
        flag = 0
    else:
        fig.add_trace(go.Bar(x=x, y=[j for i, j in sn], name=items, width=3,))
    i += 1
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

In [75]:
import sys
import json
import importlib

from typing import Iterator
from typing import List
from typing import Optional
from typing import Tuple

import pandas as pd

from scipy.sparse import csc_matrix
from scipy.spatial.distance import euclidean
from sklearn.decomposition import TruncatedSVD

from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from pyclustering.cluster import xmeans

In [76]:
def _get_s_vecs(
        doc_vecs: Iterator[List[Tuple[int, float]]],
        shape: Optional[Tuple[int, int]] = None) -> csc_matrix:
    data, row_inds, col_inds = [], [], []
    for row_ind, doc_vec in enumerate(doc_vecs):
        if not doc_vec:
            continue

        for wid, val in doc_vec:
            row_inds.append(row_ind)
            col_inds.append(wid)
            data.append(val)

    return csc_matrix((data, (row_inds, col_inds)), shape=shape)

In [77]:
df = df.loc[df['pro'].notnull(), :]

# 単語分解
words_list = []
for _idx, row in df.iterrows():
    in_str = row['pro']
    words_list.append(in_str.split())

In [ ]:
words_list

In [78]:
# 辞書作成、単語フィルタリング
dic = Dictionary(words_list)
dic.filter_extremes(no_below=3, no_above=0.3)

# tf-idfエンコーディング
bows = list(map(dic.doc2bow, words_list))
tfidf_model = TfidfModel(bows)
s_vecs = _get_s_vecs(tfidf_model[bows])

trunc_svd_model = TruncatedSVD(n_components=30, random_state=0)
trunc_svd_model.fit(s_vecs)
comp_vecs = trunc_svd_model.transform(s_vecs)

initializer = xmeans.kmeans_plusplus_initializer(
    data=comp_vecs, amount_centers=25)
init_centers = initializer.initialize()
xm_model = xmeans.xmeans(data=comp_vecs, initial_centers=init_centers, kmax=50, tolerance=0.0001, criterion=0)
xm_model.process()
cl_labels = xm_model.predict(comp_vecs)
df['cluster label'] = cl_labels


In [79]:
centers = xm_model.get_centers()
df['distance from cluster center'] = list(map(
    lambda x: euclidean(centers[x[0]], x[1]),
    zip(cl_labels, comp_vecs)))

# 結果格納
df = df.sort_values(by=['cluster label', 'distance from cluster center'])

In [80]:
df

,pro,raw,cluster label,distance from cluster center
332,dポイント 使い道,dポイント 使い道,0,0.049476
4930,dポイント 使い道,dポイント 使い道,0,0.049476
5015,dポイント 使い道,dポイント 使い道,0,0.049476
5098,dポイント 使い道,dポイント 使い道,0,0.049476
7260,dポイント 使い道,dポイント 使い道,0,0.049476
...,...,...,...,...
14004,docomo 提供 サービス 支払い,ドコモが提供するサービスでは、どのように支払いをするのですか？,49,0.056805
15587,docomo 提供 サービス 支払い,ドコモが提供するサービスでは、どのように支払いをするのですか？,49,0.056805
16339,docomo 提供 サービス 支払い,ドコモが提供するサービスでは、どのように支払いをするのですか？,49,0.056805
14702,オリックス 返却 ステーション クルマ 質問 カーシェア dカーシェア docomo カーシ...,オリックスで返却する際に、ステーションに他のクルマが停まっていた。どうすればいいですか？ |...,49,0.336464


In [70]:
df[df['cluster label']==0]

,pro,raw,cluster label,distance from cluster center
6323,dポイント 使い方,dポイントの使い方は？,0,0.040097
6557,dポイント 使い方,dポイントの使い方は？,0,0.040097
9463,dポイント 使い方,dポイントの使い方は？,0,0.040097
11106,dポイント 使い方,dポイントの使い方は？,0,0.040097
14797,dポイント 使い方,dポイントの使い方は？,0,0.040097
...,...,...,...,...
7900,dアカウント 発行 必須,ｄアカウントの発行は必須か,0,0.457379
10962,dアカウント 発行 必須,ｄアカウントの発行は必須か,0,0.457379
105,dアカウント 解約 共同使用料,ｄアカウントを解約したが共同使用料は受け取れる？,0,0.476678
106,dアカウント 解約 共同使用料,ｄアカウントを解約したが共同使用料は受け取れる？,0,0.476678


In [103]:
df['text'] = df['raw']

In [83]:
def cluster_stat(record):
    record = record.groupby('cluster label')['raw'].count().reset_index(name="Count")
    record = record.sort_values(by='Count', ascending=False)
    total = sum(record['Count'].values.tolist())
    record['Percent_%'] = record.Count.apply(lambda x: x / total * 100)
    return record

In [97]:
def get_cluster_sentences(record, cluster_index=0):
    return record[(record['cluster_ID'] == cluster_index)].text.values.tolist()

In [84]:
r = cluster_stat(df)

In [92]:
r.columns

Index(['cluster label', 'Count', 'Percent_%'], dtype='object')

In [96]:
# import plotly.express as px
# import numpy as np

import plotly.express as px
fig = px.bar(r, x='cluster label', y='Percent_%', labels={'Percent_%':'Cluster Population %'}, color='Count')
fig.show()

In [105]:
from collections import Counter
sn = Counter(get_cluster_sentences(df, 2)).most_common()

In [109]:
s = []
c = []
for i, j in sn:
    s.append(i)
    c.append(j)

In [110]:
import plotly.graph_objects as go

x=s
fig = go.Figure(go.Bar(x=x, y=c, name='Montreal'))
fig.add_trace(go.Bar(x=x, y=[1, 4, 9, 16], name='Ottawa'))
fig.add_trace(go.Bar(x=x, y=[6, 8, 4.5, 8], name='Toronto'))

fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()